In [1]:
import pandas as pd
import time
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession, types
import pyspark.sql.functions as F
from  pyspark.sql.functions import col
data_filepath = "../data/steam_reviews.csv"

In [2]:
steam_reviews_schema = types.StructType([
    types.StructField('_c0', types.StringType()),
    types.StructField('app_id', types.StringType()),
    types.StructField('app_name', types.StringType()),
    types.StructField('review_id', types.StringType()),
    types.StructField('language', types.StringType()),
    types.StructField('review', types.StringType()),
    types.StructField('timestamp_created', types.StringType()),
    types.StructField('timestamp_updated', types.StringType()),
    types.StructField('recommended', types.BooleanType()),
    types.StructField('votes_helpful', types.IntegerType()),
    types.StructField('votes_funny', types.IntegerType()),
    types.StructField('weighted_vote_score', types.FloatType()),
    types.StructField('comment_count', types.IntegerType()),
    types.StructField('steam_purchase', types.BooleanType()),
    types.StructField('received_for_free', types.BooleanType()),
    types.StructField('written_during_early_access', types.BooleanType()),
    types.StructField('author.steamid', types.StringType()),
    types.StructField('author.num_games_owned', types.IntegerType()),
    types.StructField('author.num_reviews', types.IntegerType()),
    types.StructField('author.playtime_forever', types.FloatType()),
    types.StructField('author.playtime_last_two_weeks', types.FloatType()),
    types.StructField('author.playtime_at_review', types.FloatType()),
    types.StructField('author.last_played', types.FloatType()),
])


In [3]:
#spark = SparkSession.builder.appName('games').getOrCreate()
spark = (
    SparkSession.builder.config("spark.sql.debug.maxToStringFields", 100)
    .appName("reviews")
    .getOrCreate()
)

steam_reviews = spark.read.format('csv') \
                    .schema(steam_reviews_schema) \
                    .option('header', 'true') \
                    .load(data_filepath)

# steam_reviews.printSchema()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/30 15:21:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
originalSize = steam_reviews.count()

steam_reviews = steam_reviews.where(col("app_id").isNotNull() \
                                    & col("app_name").isNotNull() \
                                    & (col("app_id") != "0") \
                                    & (col("app_name") != "0") \
                                    & (col("app_name").cast("int").isNull()) \
                                    & (col("app_id").cast("int").isNotNull()) \
                                    & (col("app_id").cast("int") != 0)  )

steam_reviews = steam_reviews.withColumnRenamed("author.steamid", "author_steamid") \
                             .withColumnRenamed("author.num_games_owned", "author_num_games_owned") \
                             .withColumnRenamed("author.num_reviews", "author_num_reviews") \
                             .withColumnRenamed("author.playtime_forever", "author_playtime_forever") \
                             .withColumnRenamed("author.playtime_last_two_weeks", "author_playtime_last_two_weeks") \
                             .withColumnRenamed("author.playtime_at_review", "author_playtime_at_review") \
                             .withColumnRenamed("author.last_played", "author_last_played")

steam_reviews = steam_reviews.drop("_c0")

all_games = steam_reviews.select(col("app_id"), col("review_id")) \
                         .groupBy(col("app_id")).agg(F.count("review_id").alias("review_count")) \
                         .where(col("review_count") >= 500) \
                         .select("app_id")

steam_reviews = steam_reviews.join(all_games.hint("broadcast"), ["app_id"])

currentSize = steam_reviews.count()
gameCount = all_games.count()

print("Number of reviews:")
print("  Original data size:", originalSize)
print("  Current data size:", currentSize)
print("  Data Removed:", originalSize - currentSize)
print("")
print("Number of games:", gameCount)
print("")

steam_reviews.show(100)

steam_reviews.write.csv("output", mode='overwrite')



Number of reviews:
  Original data size: 40848659
  Current data size: 21747175
  Data Removed: 19101484

Number of games: 314



+------+--------------------+---------+----------+-------------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
|app_id|            app_name|review_id|  language|                               review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|
+------+--------------------+---------+----------+-------------------------------------+-----------------+-----------------+-----------+-------------+-----------+--

In [10]:
all_games = steam_reviews.select(col("app_id"), col("review_id")) \
                         .groupBy(col("app_id")).agg(F.count("review_id").alias("review_count")) \
                         .where(col("review_count") >= 500) \
                         .select("app_id")
                        #  .orderBy(col("review_count"))

In [11]:
steam_reviews = steam_reviews.join(all_games.hint("broadcast"), ["app_id"])

In [12]:
test.show(100)

+------+--------------------+---------+----------+-------------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
|app_id|            app_name|review_id|  language|                               review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|
+------+--------------------+---------+----------+-------------------------------------+-----------------+-----------------+-----------+-------------+-----------+--

In [13]:
afterSize = test.count()


print("Original data size:", originalSize)
print("Current data size:", afterSize)
print("Total Removed:", originalSize - afterSize)
print("Little Game Removed:", currentSize - afterSize)

Original data size: 40848659
Current data size: 21747175
Total Removed: 19101484
Little Game Removed: 2361


In [8]:
test = steam_reviews.where(col("app_id") == "619290")
test.show(100)

+------+--------------------+---------+---------+-----------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
|app_id|            app_name|review_id| language|                             review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|
+------+--------------------+---------+---------+-----------------------------------+-----------------+-----------------+-----------+-------------+-----------+-----------

In [4]:
steam_reviews.show(100)

+--------------------+--------------------+--------------------+--------------------+-------------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
|              app_id|            app_name|           review_id|            language|                               review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|
+--------------------+--------------------+--------------------+--------------------+---------

# Ignore

In [ ]:
unwanted = steam_reviews.select(col("app_id")).where(col("app_id").isNull() \
                              | (F.lower(col("app_id")) == "null") \
                              | (col("app_id") == "") \
                              | (col("app_id") == "0"))

In [5]:
unwanted.show(10)

+------+
|app_id|
+------+
+------+



In [6]:
unwanted = steam_reviews.select(col("app_name")).where(col("app_name").isNull() \
                              | (F.lower(col("app_name")) == "null") \
                              | (col("app_name") == "") \
                              | (col("app_name") == "0"))

In [7]:
unwanted.show(10)

+--------+
|app_name|
+--------+
+--------+

